In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import imageio

from scipy.optimize import basinhopping
from scipy.ndimage import median_filter

from pymedphys.film import calc_calibration_points, load_cal_scans, create_dose_function, calc_net_od

In [ ]:
data_dir = Path('../../packages/pymedphys_analysis/tests/film/data/spine_case/DatasetB')

In [ ]:
prescans_dir = data_dir.joinpath('prescans/calibration')
postscans_dir = data_dir.joinpath('postscans/calibration')

In [ ]:
treatment_prescan = data_dir.joinpath('prescans/treatment.tif')
treatment_postscan = data_dir.joinpath('postscans/treatment.tif')

In [ ]:
prescans = load_cal_scans(prescans_dir)
postscans = load_cal_scans(postscans_dir)

In [ ]:
alignments = {650.0: (-0.03693119837922606, -0.0947635537173132, -0.37783221426724356),
 200.0: (-0.10040063983494804, -0.12169175795144278, 0.08348855748332341),
 1250.0: (-0.3032988411889664, 0.03641656353978792, 0.18968972666517137),
 1000.0: (0.26492510731092084, 0.14060820397579227, -2.7193448468631343),
 400.0: (-0.08179282402379366, -0.0679576959847175, 0.3154216316864221),
 800.0: (0.18816305442327136, 0.07207526506185861, -0.9586838845463561),
 0.0: (-0.09484272563570335, 0.12621856065409, -0.7927960758287966),
 550.0: (-0.21896673466613484, -0.3278034483394189, -2.275540688446474),
 300.0: (-0.06648768945080456, -0.0075448105213691595, -0.4383480611779579),
 1400.0: (0.21542726392752917, 0.2342317915525293, 0.031818297008679844),
 700.0: (0.04596862036224883, -0.1862675492795798, -0.5631021348448653),
 620.0: (0.045973899659790485, 0.008418383396753638, 0.056398167284774),
 500.0: (0.0157791792147568, 0.14108835739889178, -0.7097770417159057),
 900.0: (-0.1973165785177423, 0.06716167826823238, 0.09585676472845489),
 600.0: (0.0655280798391002, 0.010254562005567824, -0.04182894414171261),
 750.0: (-0.13149497704528673, 0.08891070049242197, 1.0078733699651568),
 580.0: (0.09292906883636731, -0.18062150972708307, -0.3395879590451904),
 1100.0: (-0.05951520624559155, -0.1508183573554567, 0.2423935149180743)}

# alignments = None  # Uncomment this line to recalc the alignments (slow)

In [ ]:
points, alignments = calc_calibration_points(prescans, postscans, alignments=alignments)
# alignments

In [ ]:
dose_cal = []
net_od_cal = []

for key, item in points.items():
    net_od_cal.append(item)
    dose_cal.append(key)

In [ ]:
index = np.argsort(net_od_cal)
dose_cal = np.array(dose_cal)[index]
net_od_cal = np.array(net_od_cal)[index]

In [ ]:
dose_function = create_dose_function(net_od_cal, dose_cal)

In [ ]:
x = np.linspace(np.min(net_od_cal), np.max(net_od_cal))
y = dose_function(x)

plt.plot(net_od_cal, dose_cal, '.')
plt.plot(x, y)

In [ ]:
treatment_prescan = imageio.imread(treatment_prescan)
treatment_postscan = imageio.imread(treatment_postscan)

In [ ]:
treatment_alignment = (-0.09021456658200089, -0.1541382806538466, -0.07098367045728018)

# treatment_alignment = None  # Uncomment this line to recalc the alignment (slow)

In [ ]:
treatment_net_od, treatment_alignment = calc_net_od(
    treatment_prescan, treatment_postscan, alignment=treatment_alignment)
# treatment_alignment

In [ ]:
treatment_dose = dose_function(treatment_net_od)

make_nan = treatment_dose > np.max(dose_cal)*1.2
treatment_dose[make_nan] = np.nan

In [ ]:
plt.figure(figsize=(10,10))
plt.pcolormesh(treatment_dose)
plt.colorbar()
plt.axis('equal')

In [ ]:
filtered_dose = median_filter(treatment_dose, size=10)

In [ ]:
plt.figure(figsize=(10,10))
plt.pcolormesh(filtered_dose)
plt.colorbar()
plt.axis('equal')